In [26]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from konlpy.tag import Mecab
from tqdm import tqdm
from collections import Counter
import itertools # 2차원 배열 1차원 배열로
from sklearn.feature_extraction.text import TfidfVectorizer
import os #폴더 이름 가져오기

## 파일 통합

In [27]:
#forders=os.listdir('/Users/home/저점')

In [28]:
#df_all=pd.DataFrame()
#for i in range(0,len(forders)):
#    if forders[i].split('.')[-1]=='csv':
#        file='/Users/home/저점/'+forders[i]
#        df=pd.read_csv(file,encoding='utf-8')
#        df_all=pd.concat([df_all,df])

In [29]:
#df=df_all
#df

## 폴더 가져오기

In [155]:
# 고점 or 저점 파일을 파라미터 값으로 넣어 지정돤 폴더에서 csv파일 리스트들을 가져오는 함수
# 파라미터 : 절대경로에 있는 파일
def choice_file(choice):
    file_list=os.listdir('/Users/home/'+choice)
    cnt=0
    for i in file_list:
        i="/Users/home/"+choice+"/"+i
        file_list[cnt]=i
        cnt+=1
    return file_list

file_list=choice_file('저점')
file_list

['/Users/home/저점/2022.11.21.csv',
 '/Users/home/저점/2022.01.22.csv',
 '/Users/home/저점/2022.09.21.csv',
 '/Users/home/저점/2022.12.28.csv',
 '/Users/home/저점/2022.10.11.csv',
 '/Users/home/저점/2022.05.27.csv',
 '/Users/home/저점/2021.10.27.csv',
 '/Users/home/저점/2022.02.21.csv',
 '/Users/home/저점/2022.09.06.csv',
 '/Users/home/저점/.ipynb_checkpoints',
 '/Users/home/저점/2022.06.18.csv']

## 필요한 데이터로 변경 : 결측치 제거/ column 조정

In [255]:
df_1=pd.read_csv(file_list[10],encoding="UTF-8") # 크롤링한 파일 가져오기

In [256]:
def delete(df):
    df=df.replace("- dc official App","") # 글 내용없을때 들어가는 "- dc official App" 대체
    df=df.fillna('') # 결측치 제거
    df['data']=df['title']+ " "+df['content'] # 제목+내용을 하나로 합쳐서 'data'로 저장 
    del df['title'],df['content'],df['no'] # 제목,내용,글번호 삭제
    data=df['data']
    return df

df=delete(df_1)
df

,date,data
0,2022.06.18 00:00:43,러시아 휴전빔 웨ㅇ브ㅅㅅ ㅎ ㅇ
1,2022.06.18 00:00:43,웨이브 극락가는데 나락간다고 헛소리하는 놈 없제 ㅋㅋ? 개미털기에 매번 속은 과테말...
2,2022.06.18 00:01:19,이럴거면 왜올렸어 나스닥빔ㅋㅋ 이씨발년아 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
3,2022.06.18 00:01:51,애미 시발련아 이게 맞아?
4,2022.06.18 00:02:19,그냥 2배롱하면 되는거아님? 10k 까지 떨어지면 나만 자살하는건 아닐테니까 2배롱...
...,...,...
9015,2022.06.19 08:59:43,라이트 상폐빔간다 잘봐라
9016,2022.06.19 08:59:47,대불장은 니미ㅋㅋ아직도 정신을 못차리네 ㅋㅋㅋㅋ
9017,2022.06.19 08:59:52,코인충 자살 도대체 몇번이냐? 또 코인충이냐?
9018,2022.06.19 08:59:53,시아코인 쏜다@@ 굿


In [257]:
# 기본적인 전처리 과정을 진행하는 함수 
# 파라미터: pandas데이터프레임의 전처리를 원하는 컬럼
def data_processing(data_column):
    data_column=df['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규화
    data_column=df['data'].str.replace('^ +', "") # 공백은 empty값으로 변경
    data_column.replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
    data_column.dropna(how='any') # Null 값 제거

data_processing(df['data'])

/var/folders/mf/zx0plt5520l4g1dfsl85d0gm0000gn/T/ipykernel_88973/3358323079.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data_column=df['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규화
/var/folders/mf/zx0plt5520l4g1dfsl85d0gm0000gn/T/ipykernel_88973/3358323079.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  data_column=df['data'].str.replace('^ +', "") # 공백은 empty값으로 변경


## 불용어 파일 리스트로 저장

In [258]:
f=open('stopwords.txt',encoding='utf-8') # 불용어 파일 
words=[]

for i in f:
    words.append(i)
f.close()

words=str(words)[2:-2]

stopwords=words.split(' ')

## 전처리 

In [259]:
m=Mecab()
result=[]
for i in tqdm(df['data']):
    tokenized_sentence =m.morphs(str(i))
    stopwords_removed_sentence=[word for word in tokenized_sentence if not word in stopwords]
    result.append(stopwords_removed_sentence)    

 59%|█████████████████████▉               | 5350/9020 [00:01<00:01, 3555.49it/s]

## 한글자는 제외

In [260]:
one_word_remove=[]
for word in result:
    tmp=[]
    for i in word:
        if len(i)>1:
            tmp.append(i)
    one_word_remove.append(tmp)
            

In [261]:
# rs=[]
# for word in tqdm(one_word_remove):
#     tmp=[]
#     for i in word:
#         if m.pos(i)[0][1]=='NNG' or m.pos(i)[0][1]=='NNP'or m.pos(i)[0][1]=='VCP' or m.pos(i)[0][1]=='VCN' or m.pos(i)[0][1]=='VV' or m.pos(i)[0][1]=='MAG' or m.pos(i)[0][1]=='VA':
#             tmp.append(m.pos(i)[0][0])
#     rs.append(tmp)

In [262]:
rs=[]
for word in tqdm(one_word_remove):
    tmp=[]
    for i in word:
        if m.pos(i)[0][1]!='JKS' or m.pos(i)[0][1]!='JKQ'or m.pos(i)[0][1]!='JKV' or m.pos(i)[0][1]=='JKB' or m.pos(i)[0][1]!='JKO' or m.pos(i)[0][1]!='JKG' or m.pos(i)[0][1]!='JKC' or m.pos(i)[0][1]!='JX'or m.pos(i)[0][1]!='JC':
            tmp.append(m.pos(i)[0][0])
    rs.append(tmp)

## 2차원 배열 1차원으로 변경후 데이터프레임에 저장

In [263]:
sentence=[]
for word in rs:
    result=' '.join(s for s in word)
    sentence.append(result)

In [264]:
df['result']=sentence

In [265]:
df.to_csv('/Users/home/저점조사제외전처리완료/2022.06.18.csv',encoding='utf-8')

## 빈도수 체크

In [42]:
frequent=Counter(sentence).most_common()

In [43]:
frequent

[('', 2442),
 ('솔브', 128),
 ('리플 쏜다 리플 쏜다', 64),
 ('휴먼', 34),
 ('이캐시', 18),
 ('솔브 아마존', 17),
 ('솔브 운지', 16),
 ('쏜다', 14),
 ('솔브 쏜다', 12),
 ('솔브 양전', 12),
 ('아마존', 12),
 ('솔브케어', 9),
 ('플로우', 8),
 ('누사이퍼', 8),
 ('코인', 8),
 ('에스티피', 8),
 ('코인 정신 승리', 8),
 ('깜짝 한다 !', 8),
 ('솔브 ㅇㅇ', 8),
 ('20 확실 답변 . txt 미성년자 라고 처음 떠오르 이미지 고딩 vs 먼저 생각나', 8),
 ('신세 간다 ㅇㅇ', 8),
 ('안녕 세요 선물 거래 계신 지금 시세 업비트 안녕 세요 선물 거래 계신 지금 시세 업비트 리플 1345 1350 왔 1400 한테 분계 실까요 해외 거래소 이용 코인 개인 해야 네요 구매 신분 확실 다면 입금 먼저 드릴 댓글 이나 페메 세요 선물 어서 제정신 네요 댓글 이나 텔레 Jusungpa 연락',
  8),
 ('스택', 7),
 ('솔브 나락', 7),
 ('휴먼 양전', 7),
 ('불장', 7),
 ('■■■■ 학식 고래 솔브 손절 오열 ■■■■', 7),
 ('신세 쏜다 ㅇㅇ', 7),
 ('솔브 어라', 6),
 ('누사', 6),
 ('지금', 6),
 ('간다', 6),
 ('현재 논란 문제 . GIF', 6),
 ('미쳤', 6),
 ('손실 꿔야', 6),
 ('■■■■ 학식 고래 솔브 절친 호구 새기 ■■■', 6),
 ('솔브 점마', 5),
 ('솔브 간다', 5),
 ('코스모스', 5),
 ('솔브 ~~', 5),
 ('으면 개추', 5),
 ('비트', 5),
 ('진짜', 5),
 ('댓글', 5),
 ('이캐시 쏜다', 5),
 ('배우 이름 사람', 5),
 ('솔브 개추', 5),
 ('진짜 아마존 이랑 협업 솔브', 5),
 ('솔브 달달', 4),
 ('달린다', 4),
 ('폴카

### csv 파일로 저장

In [44]:
idx=df[df['result']=='nan'].index
df.drop(idx,inplace=True)

In [45]:
corpus=list(df['result'])

In [46]:
corpus

['스택',
 '솔브 풀리',
 '이걸 올리 무빙 죶같노',
 '솔브 솔브 kucoin 146 토시 환장',
 '이랑 석우 점검 직전 더니 회복',
 '플로우 차트 지대 만드',
 '밀지 마라',
 '누사이퍼 온다 눈물 가린다',
 '도박 장문 !!!!!',
 '포지션 무빙 죶같네 비트 이건 관망 장이 들어가 물린다',
 '비트 그냥 내릴 마다 계속 늘어나 충이',
 '진심 누사 플로우 자살 아라',
 '점검 업비트 화면 멈춰 그러',
 '업비트 미동',
 '이캐시 차트 올리',
 '속보 폴리곤 1750 1700 예정',
 '점검 라고 아재 겁쟁이 그게',
 '석우 패닉 줘서 고마우 개추',
 '업비트 된다',
 '10 입금 !!!!!!!!!!!!!',
 '점검 30 빨리 설렜 도박',
 '누사 방금 사람 렸 ㅇㅇ',
 '서버 점검 2021 10 19 23 30 00 2021',
 '작업 들어갔 빨리 놔라 뺀다 거래량 올라갈 거래량 다시',
 '코스모스 시볼 까',
 '플로우 가격 100 원임 . 바위',
 '라면 끓여 스톰 엑스 36 놨는데',
 '모스',
 '거래 풀렸 ㅇㅇ',
 '코스모스 오늘 돌파 보소',
 '업비트 열렸 드가',
 '노무현 단계 코인 노무현 매매 ㅆㅅㅌㅊ던데',
 '모여 업비트 점검 거래 어서 뭔',
 '코스모스 근황 랏다 ~~~ 내렸 다가 ~~~ 올랐 다가 ~~~ 내려갔 다가 ~~ 아까 보단 지금 올랐 ㅇㅇ 그냥',
 '솔브케어 관련 박제',
 '현재 알트 이유 필독 해라 ㅇㅇ 비트코인 도미 비트 횡보 알트 요지부동 찔끔찔끔 떨어질 보일 이유 에스티피 ㅇㅇ 애별 세력 빨아먹 으려고 미친 오르내리 일반 알트 빠짐 이유 에스티피 이후 10 짜리 동전 알트 너 40 올려 개지랄 해서 세력 재미 알트 세력 빠짐 이유 누사이퍼 면서 개지랄 왔 면서 알트 세력 거래량 터짐 에스티피 누사이퍼 누적 거래량 20 터짐 와중 불나방 유입 ㅄ마냥 쳐먹 다고 달려들 다가 시체 변하 조막손 세력 물려 그러 비트 오르 다가 하루 이틀 횡보 원래 올라야 세력 

In [47]:
s=[]
for i in corpus:
    i=i.replace(" ","")
    s.append(i)

In [48]:
s

['스택',
 '솔브풀리',
 '이걸올리무빙죶같노',
 '솔브솔브kucoin146토시환장',
 '이랑석우점검직전더니회복',
 '플로우차트지대만드',
 '밀지마라',
 '누사이퍼온다눈물가린다',
 '도박장문!!!!!',
 '포지션무빙죶같네비트이건관망장이들어가물린다',
 '비트그냥내릴마다계속늘어나충이',
 '진심누사플로우자살아라',
 '점검업비트화면멈춰그러',
 '업비트미동',
 '이캐시차트올리',
 '속보폴리곤17501700예정',
 '점검라고아재겁쟁이그게',
 '석우패닉줘서고마우개추',
 '업비트된다',
 '10입금!!!!!!!!!!!!!',
 '점검30빨리설렜도박',
 '누사방금사람렸ㅇㅇ',
 '서버점검202110192330002021',
 '작업들어갔빨리놔라뺀다거래량올라갈거래량다시',
 '코스모스시볼까',
 '플로우가격100원임.바위',
 '라면끓여스톰엑스36놨는데',
 '모스',
 '거래풀렸ㅇㅇ',
 '코스모스오늘돌파보소',
 '업비트열렸드가',
 '노무현단계코인노무현매매ㅆㅅㅌㅊ던데',
 '모여업비트점검거래어서뭔',
 '코스모스근황랏다~~~내렸다가~~~올랐다가~~~내려갔다가~~아까보단지금올랐ㅇㅇ그냥',
 '솔브케어관련박제',
 '현재알트이유필독해라ㅇㅇ비트코인도미비트횡보알트요지부동찔끔찔끔떨어질보일이유에스티피ㅇㅇ애별세력빨아먹으려고미친오르내리일반알트빠짐이유에스티피이후10짜리동전알트너40올려개지랄해서세력재미알트세력빠짐이유누사이퍼면서개지랄왔면서알트세력거래량터짐에스티피누사이퍼누적거래량20터짐와중불나방유입ㅄ마냥쳐먹다고달려들다가시체변하조막손세력물려그러비트오르다가하루이틀횡보원래올라야세력올리려는매물매집했으니올림그러비트올라가니까도미빨려한주춤악순환반복알트올라가제일이유ㅇㅇ이미많이올라와서새로운차트깨세력입장에선올리부담스러움비트나락떄매올리은데비트계속오르니까매집구간어서오르ㅇㅇ이게결과지금그냥타이밍아님ㅇㅇ',
 '●●●업비트정상가동시작●●●빨리',
 '업비트움직이열린',
 '열림',
 '현재코스모스상황',
 '플로우시장던졌ㅐㅆㅣ발!손절',
 '진짜거래',
 '폴리곤들어가아까1850갑자기내

In [49]:
tfidfv=TfidfVectorizer(max_features=10).fit(s)
tfidfv.transform(corpus).toarray()
tfidfv.vocabulary_

{'솔브': 3,
 'ㅇㅇ': 1,
 '휴먼': 9,
 '이캐시': 7,
 '솔브아마존': 4,
 '영원사랑': 5,
 '우어허어': 6,
 'woar': 0,
 '지금달라질가제': 8,
 '리플쏜다리플쏜다': 2}